<a href="https://colab.research.google.com/github/Worachet-Ch/project2022/blob/main/random_nan_and_imputation_with_weight_variable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd #นำเข้า pandas แทนที่ด้วย pd
import os# os ใช้จัดการกับไฟล์ ว่าอยู่ที่ไหน
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as mno
import missingno as msno
from sklearn import linear_model
%matplotlib inline
import matplotlib
import sklearn 
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
import time

In [ ]:
from google.colab import drive #เชื่อม Google drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/project' #เข้า drive
data_file_path = os.path.join(path,'data_notmissing.csv') #ไปที่ไฟล์ confirmed-cases-since-280265.csv
data = pd.read_csv(data_file_path) #อ่านไฟล์ csv

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data.drop({'Unnamed: 0','telomere length (kb) '},1,inplace = True)

<ipython-input-146-4cc622f169ae>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data.drop({'Unnamed: 0','telomere length (kb) '},1,inplace = True)


In [ ]:
data

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,003_Creatinine,004_Uric Acid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,013_Alkaline Phos,071_HbA1c
0,2,57.0,47.4,153.0,20.2,114.0,75.0,69.0,19.3,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
1,2,24.0,53.2,157.0,21.6,103.0,69.0,78.0,21.3,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
2,2,62.0,54.4,165.0,20.0,95.0,75.0,83.0,21.6,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
3,2,35.0,56.3,161.0,21.7,95.0,57.0,72.0,22.5,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.1
4,1,46.0,72.3,170.0,25.0,133.0,77.0,107.0,29.2,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1,60.0,72.0,170.0,24.9,131.0,74.0,69.0,30.5,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.5
149,1,58.0,62.0,168.0,22.0,103.0,76.0,72.0,27.8,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.8
150,2,42.0,55.5,165.0,20.4,102.0,52.0,77.0,22.9,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.8
151,2,61.0,61.8,159.0,24.4,117.0,60.0,72.0,23.4,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.4


# 10 % random nan

In [ ]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'weight{i}'] = data.copy() # for loop variable name
  np.random.seed(i) # for loop seed
  n = int((153*10/100)) 
  index_nan = np.random.choice(globals()[f'weight{i}']['Weight'].size, n, replace=False) #globals()[name dataframe][column name].size
  globals()[f'weight{i}']['Weight'].ravel()[index_nan] = np.nan
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

เวลาที่ใช้ในการประมวลผล 0.2516813278198242 วินาที
เวลาที่ใช้ในการประมวลผล 0.00419468879699707 นาที


In [ ]:
weight1

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,003_Creatinine,004_Uric Acid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,013_Alkaline Phos,071_HbA1c
0,2,57.0,47.4,153.0,20.2,114.0,75.0,69.0,19.3,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
1,2,24.0,53.2,157.0,21.6,103.0,69.0,78.0,21.3,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
2,2,62.0,54.4,165.0,20.0,95.0,75.0,83.0,21.6,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
3,2,35.0,56.3,161.0,21.7,95.0,57.0,72.0,22.5,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.1
4,1,46.0,72.3,170.0,25.0,133.0,77.0,107.0,29.2,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1,60.0,72.0,170.0,24.9,131.0,74.0,69.0,30.5,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.5
149,1,58.0,62.0,168.0,22.0,103.0,76.0,72.0,27.8,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.8
150,2,42.0,55.5,165.0,20.4,102.0,52.0,77.0,22.9,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.8
151,2,61.0,61.8,159.0,24.4,117.0,60.0,72.0,23.4,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.4


## Imputation using Mean

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
t_start = time.time()
mean10_imputer = SimpleImputer(strategy='mean')
for i in range(1,1001):
  globals()[f'imputed_mean10_{i}'] = mean10_imputer.fit_transform(globals()[f'weight{i}'].copy()) #for loop miss forest impputation 
  globals()[f'MSE_MEAN10_{i}'] = mean_squared_error(data, globals()[f'imputed_mean10_{i}']) # for loop mean_squared_error
t_end = time.time()
t10_MEAN = (t_end- t_start)

In [ ]:
t_start = time.time()
mse_mean10_total = 0 
MSE_MEAN10_1001 = 0
for i in range(0,1001):
  mse_mean10_total = mse_mean10_total + globals()[f'MSE_MEAN10_{i+1}']
AMSE_weight10_mean = mse_mean10_total/1000
print(f'AMSE_weight10_mean = {AMSE_weight10_mean}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight10_mean = 0.2982275919348313
เวลาที่ใช้ในการประมวลผล 0.0011734962463378906 วินาที
เวลาที่ใช้ในการประมวลผล 1.9558270772298177e-05 นาที


## Imputation using the miss forest

In [ ]:
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
!pip install missingpy
import missingpy
from missingpy import MissForest
from sklearn.metrics import mean_squared_error

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
t_start = time.time()
MF10_imputer = MissForest()
for i in range(1,1001):
  globals()[f'imputed_mf10_{i}'] = MF10_imputer.fit_transform(globals()[f'weight{i}'].copy()) #for loop miss forest impputation 
  globals()[f'MSE_MF10_{i}'] = mean_squared_error(data, globals()[f'imputed_mf10_{i}']) # for loop mean_squared_error
t_end = time.time()
t10_MF= (t_end- t_start)

In [ ]:
t_start = time.time()
mse_mf10_total = 0 
MSE_MF10_1001 = 0
for i in range(0,1001):
  mse_mf10_total = mse_mf10_total + globals()[f'MSE_MF10_{i+1}']
AMSE_weight10_mf = mse_mf10_total/1000
print(f'AMSE_weight10_mf = {AMSE_weight10_mf}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight10_mf = 0.030136010330568137
เวลาที่ใช้ในการประมวลผล 0.008789777755737305 วินาที
เวลาที่ใช้ในการประมวลผล 0.0001464962959289551 นาที


## Imputation using the KNN

In [ ]:
from sklearn import neighbors
from sklearn.impute import KNNImputer

In [ ]:
t_start = time.time()
KNN10_imputer = KNNImputer(n_neighbors = 15)
for i in range(1,1001):
  globals()[f'imputed_KNN10_{i}'] = KNN10_imputer.fit_transform(globals()[f'weight{i}'].copy())
  globals()[f'MSE_KNN10_{i}'] = mean_squared_error(data, globals()[f'imputed_KNN10_{i}'])
t_end = time.time()
t10_KNN= (t_end- t_start)

In [ ]:
t_start = time.time()
mse_knn10_total = 0 
MSE_KNN10_1001 = 0
for i in range(0,1001):
  mse_knn10_total = mse_knn10_total + globals()[f'MSE_KNN10_{i+1}']
AMSE_weight10_knn = mse_knn10_total/1000
print(f'AMSE_weight10_knn = {AMSE_weight10_knn}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight10_knn = 0.1805482678733033
เวลาที่ใช้ในการประมวลผล 0.003055572509765625 วินาที
เวลาที่ใช้ในการประมวลผล 5.092620849609375e-05 นาที


## Imputation using the MICE

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn import svm

### LinearRegression model

In [ ]:
t_start = time.time()
lr = LinearRegression()
for i in range(1,1001):
  imp = IterativeImputer(estimator= lr , max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_lr10_{i}'] = imp.fit_transform(globals()[f'weight{i}'].copy())
  globals()[f'MSE_lr10_{i}'] = mean_squared_error(data, globals()[f'imputed_lr10_{i}'])
t_end = time.time()
t10_LR = (t_end- t_start)

In [ ]:
t_start = time.time()
mse_lr10_total = 0 
MSE_lr10_1001 = 0
for i in range(0,1001):
  mse_lr10_total = mse_lr10_total + globals()[f'MSE_lr10_{i+1}']
AMSE_weight10_lr = mse_lr10_total/1000
print(f'AMSE_weight10_lr = {AMSE_weight10_lr}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight10_lr = 0.004184412666016818
เวลาที่ใช้ในการประมวลผล 0.005227565765380859 วินาที
เวลาที่ใช้ในการประมวลผล 8.712609608968099e-05 นาที


### Support Vector Machine model

In [ ]:
t_start = time.time()
regr = svm.SVR()
for i in range(1,1001):
  svm_svr = IterativeImputer(estimator= regr, max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_svm10_{i}'] = svm_svr.fit_transform(globals()[f'weight{i}'].copy())
  globals()[f'MSE_svm10_{i}'] = mean_squared_error(data, globals()[f'imputed_svm10_{i}'])
t_end = time.time()
t10_SVM = (t_end- t_start)

In [ ]:
t_start = time.time()
mse_svm10_total = 0 
MSE_svm10_1001 = 0
for i in range(0,1001):
  mse_svm10_total = mse_svm10_total + globals()[f'MSE_svm10_{i+1}']
AMSE_weight10_svm = mse_svm10_total/1000
print(f'AMSE_weight10_svm = {AMSE_weight10_svm}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight10_svm = 0.24436632066858974
เวลาที่ใช้ในการประมวลผล 0.0033721923828125 วินาที
เวลาที่ใช้ในการประมวลผล 5.620320638020833e-05 นาที


## Imputation using the EM

### impy.em

In [ ]:
!pip install impyute
import impyute as impy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'imputed_impy10_{i}'] = impy.em(globals()[f'weight{i}'].values.copy())
  globals()[f'MSE_impy10_{i}'] = mean_squared_error(data, globals()[f'imputed_impy10_{i}'])
t_end = time.time()
t10_EM = (t_end- t_start)

In [ ]:
t_start = time.time()
mse_impy10_total = 0 
MSE_impy10_1001 = 0
for i in range(0,1001):
  mse_impy10_total = mse_impy10_total + globals()[f'MSE_impy10_{i+1}']
AMSE_weight10_impy = mse_impy10_total/1000
print(f'AMSE_weight10_impy = {AMSE_weight10_impy}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight10_impy = 0.5530573965089465
เวลาที่ใช้ในการประมวลผล 0.0030622482299804688 วินาที
เวลาที่ใช้ในการประมวลผล 5.103747049967448e-05 นาที


# 20 % random nan

In [ ]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'weight20_{i}'] = data.copy() # for loop variable name
  np.random.seed(i) # for loop seed
  n = int((153*20/100)) 
  index_nan = np.random.choice(globals()[f'weight20_{i}']['Weight'].size, n, replace=False) #globals()[name dataframe][column name].size
  globals()[f'weight20_{i}']['Weight'].ravel()[index_nan] = np.nan
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

เวลาที่ใช้ในการประมวลผล 0.6687350273132324 วินาที
เวลาที่ใช้ในการประมวลผล 0.011145583788553874 นาที


In [ ]:
weight20_1

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,003_Creatinine,004_Uric Acid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,013_Alkaline Phos,071_HbA1c
0,2,57.0,47.4,153.0,20.2,114.0,75.0,69.0,19.3,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
1,2,24.0,53.2,157.0,21.6,103.0,69.0,78.0,21.3,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
2,2,62.0,54.4,165.0,20.0,95.0,75.0,83.0,21.6,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
3,2,35.0,56.3,161.0,21.7,95.0,57.0,72.0,22.5,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.1
4,1,46.0,72.3,170.0,25.0,133.0,77.0,107.0,29.2,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1,60.0,72.0,170.0,24.9,131.0,74.0,69.0,30.5,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.5
149,1,58.0,62.0,168.0,22.0,103.0,76.0,72.0,27.8,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.8
150,2,42.0,55.5,165.0,20.4,102.0,52.0,77.0,22.9,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.8
151,2,61.0,61.8,159.0,24.4,117.0,60.0,72.0,23.4,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.4


## Imputation using Mean

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
t_start = time.time()
mean20_imputer = SimpleImputer(strategy='mean')
for i in range(1,1001):
  globals()[f'imputed_mean20_{i}'] = mean10_imputer.fit_transform(globals()[f'weight20_{i}'].copy()) #for loop miss forest impputation 
  globals()[f'MSE_MEAN20_{i}'] = mean_squared_error(data, globals()[f'imputed_mean20_{i}']) # for loop mean_squared_error
t_end = time.time()
t20_MEAN = (t_end- t_start)

In [ ]:
t_start = time.time()
mse_mean20_total = 0 
MSE_MEAN20_1001 = 0
for i in range(0,1001):
  mse_mean20_total = mse_mean20_total + globals()[f'MSE_MEAN20_{i+1}']
AMSE_weight20_mean = mse_mean20_total/1000
print(f'AMSE_weight20_mean = {AMSE_weight20_mean}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight20_mean = 0.5866671506272794
เวลาที่ใช้ในการประมวลผล 0.0036537647247314453 วินาที
เวลาที่ใช้ในการประมวลผล 6.0896078745524086e-05 นาที


## Imputation using the miss forest

In [ ]:
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
!pip install missingpy
import missingpy
from missingpy import MissForest
from sklearn.metrics import mean_squared_error

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
t_start = time.time()
MF20_imputer = MissForest()
for i in range(1,1001):
  globals()[f'imputed_mf20_{i}'] = MF20_imputer.fit_transform(globals()[f'weight20_{i}'].copy()) #for loop miss forest impputation 
  globals()[f'MSE_MF20_{i}'] = mean_squared_error(data, globals()[f'imputed_mf20_{i}']) # for loop mean_squared_error
t_end = time.time()
t20_MF = (t_end- t_start)

In [ ]:
t_start = time.time()
mse_mf20_total = 0 
MSE_MF20_1001 = 0
for i in range(0,1001):
  mse_mf20_total = mse_mf20_total + globals()[f'MSE_MF20_{i+1}']
AMSE_weight20_mf = mse_mf20_total/1000
print(f'AMSE_weight20_mf = {AMSE_weight20_mf}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight20_mf = 0.06149018353770736
เวลาที่ใช้ในการประมวลผล 0.005559682846069336 วินาที
เวลาที่ใช้ในการประมวลผล 9.266138076782226e-05 นาที


## Imputation using the KNN

In [ ]:
from sklearn import neighbors
from sklearn.impute import KNNImputer

In [ ]:
t_start = time.time()
KNN20_imputer = KNNImputer(n_neighbors = 15)
for i in range(1,1001):
  globals()[f'imputed_KNN20_{i}'] = KNN10_imputer.fit_transform(globals()[f'weight20_{i}'].copy())
  globals()[f'MSE_KNN20_{i}'] = mean_squared_error(data, globals()[f'imputed_KNN20_{i}'])
t_end = time.time()
t20_KNN = (t_end- t_start)

In [ ]:
t_start = time.time()
mse_knn20_total = 0 
MSE_KNN20_1001 = 0
for i in range(0,1001):
  mse_knn20_total = mse_knn20_total + globals()[f'MSE_KNN20_{i+1}']
AMSE_weight20_knn = mse_knn20_total/1000
print(f'AMSE_weight20_knn = {AMSE_weight20_knn}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight20_knn = 0.3633119561700464
เวลาที่ใช้ในการประมวลผล 0.013718843460083008 วินาที
เวลาที่ใช้ในการประมวลผล 0.00022864739100138346 นาที


## Imputation using the MICE

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn import svm

### LinearRegression model

In [ ]:
t_start = time.time()
lr = LinearRegression()
for i in range(1,1001):
  imp20 = IterativeImputer(estimator= lr , max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_lr20_{i}'] = imp20.fit_transform(globals()[f'weight20_{i}'].copy())
  globals()[f'MSE_lr20_{i}'] = mean_squared_error(data, globals()[f'imputed_lr20_{i}'])
t_end = time.time()
t20_LR = (t_end- t_start)

In [ ]:
t_start = time.time()
mse_lr20_total = 0 
MSE_lr20_1001 = 0
for i in range(0,1001):
  mse_lr20_total = mse_lr20_total + globals()[f'MSE_lr20_{i+1}']
AMSE_weight20_lr = mse_lr20_total/1000
print(f'AMSE_weight20_lr = {AMSE_weight20_lr}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight20_lr = 0.011561803266590068
เวลาที่ใช้ในการประมวลผล 0.0020143985748291016 วินาที
เวลาที่ใช้ในการประมวลผล 3.3573309580485025e-05 นาที


### Support Vector Machine model

In [ ]:
t_start = time.time()
regr = svm.SVR()
for i in range(1,1001):
  svm_svr = IterativeImputer(estimator= regr, max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_svm20_{i}'] = svm_svr.fit_transform(globals()[f'weight20_{i}'].copy())
  globals()[f'MSE_svm20_{i}'] = mean_squared_error(data, globals()[f'imputed_svm20_{i}'])
t_end = time.time()
t20_SVM = (t_end- t_start)

In [ ]:
t_start = time.time()
mse_svm20_total = 0 
MSE_svm20_1001 = 0
for i in range(0,1001):
  mse_svm20_total = mse_svm20_total + globals()[f'MSE_svm20_{i+1}']
AMSE_weight20_svm = mse_svm20_total/1000
print(f'AMSE_weight10_svm = {AMSE_weight20_svm}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight10_svm = 0.49041936875791103
เวลาที่ใช้ในการประมวลผล 0.0028231143951416016 วินาที
เวลาที่ใช้ในการประมวลผล 4.705190658569336e-05 นาที


## Imputation using the EM

### impy.em

In [ ]:
!pip install impyute
import impyute as impy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'imputed_impy20_{i}'] = impy.em(globals()[f'weight20_{i}'].values.copy())
  globals()[f'MSE_impy20_{i}'] = mean_squared_error(data, globals()[f'imputed_impy20_{i}'])
t_end = time.time()
t20_EM = (t_end- t_start)

In [ ]:
t_start = time.time()
mse_impy20_total = 0 
MSE_impy20_1001 = 0
for i in range(0,1001):
  mse_impy20_total = mse_impy20_total + globals()[f'MSE_impy20_{i+1}']
AMSE_weight20_impy = mse_impy20_total/1000
print(f'AMSE_weight20_impy = {AMSE_weight20_impy}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight20_impy = 1.0935381960555157
เวลาที่ใช้ในการประมวลผล 0.004238128662109375 วินาที
เวลาที่ใช้ในการประมวลผล 7.063547770182291e-05 นาที


# 30 % random nan

In [ ]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'weight30_{i}'] = data.copy() # for loop variable name
  np.random.seed(i) # for loop seed
  n = int((153*30/100)) 
  index_nan = np.random.choice(globals()[f'weight30_{i}']['Weight'].size, n, replace=False) #globals()[name dataframe][column name].size
  globals()[f'weight30_{i}']['Weight'].ravel()[index_nan] = np.nan
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

เวลาที่ใช้ในการประมวลผล 0.25695323944091797 วินาที
เวลาที่ใช้ในการประมวลผล 0.004282553990681966 นาที


In [ ]:
weight30_1

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,003_Creatinine,004_Uric Acid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,013_Alkaline Phos,071_HbA1c
0,2,57.0,47.4,153.0,20.2,114.0,75.0,69.0,19.3,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
1,2,24.0,53.2,157.0,21.6,103.0,69.0,78.0,21.3,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
2,2,62.0,54.4,165.0,20.0,95.0,75.0,83.0,21.6,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
3,2,35.0,56.3,161.0,21.7,95.0,57.0,72.0,22.5,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.1
4,1,46.0,72.3,170.0,25.0,133.0,77.0,107.0,29.2,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1,60.0,72.0,170.0,24.9,131.0,74.0,69.0,30.5,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.5
149,1,58.0,62.0,168.0,22.0,103.0,76.0,72.0,27.8,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.8
150,2,42.0,55.5,165.0,20.4,102.0,52.0,77.0,22.9,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.8
151,2,61.0,61.8,159.0,24.4,117.0,60.0,72.0,23.4,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.4


## Imputation using Mean

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
t_start = time.time()
mean30_imputer = SimpleImputer(strategy='mean')
for i in range(1,1001):
  globals()[f'imputed_mean30_{i}'] = mean30_imputer.fit_transform(globals()[f'weight30_{i}'].copy()) #for loop miss forest impputation 
  globals()[f'MSE_MEAN30_{i}'] = mean_squared_error(data, globals()[f'imputed_mean30_{i}']) # for loop mean_squared_error
t_end = time.time()
t30_MEAN = (t_end- t_start)

In [ ]:
t_start = time.time()
mse_mean30_total = 0 
MSE_MEAN30_1001 = 0
for i in range(0,1001):
  mse_mean30_total = mse_mean30_total + globals()[f'MSE_MEAN30_{i+1}']
AMSE_weight30_mean = mse_mean30_total/1000
print(f'AMSE_weight30_mean = {AMSE_weight30_mean}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight30_mean = 0.8847872255080415
เวลาที่ใช้ในการประมวลผล 0.011403560638427734 วินาที
เวลาที่ใช้ในการประมวลผล 0.00019005934397379556 นาที


## Imputation using the miss forest

In [ ]:
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
!pip install missingpy
import missingpy
from missingpy import MissForest
from sklearn.metrics import mean_squared_error

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
t_start = time.time()
MF30_imputer = MissForest()
for i in range(1,1001):
  globals()[f'imputed_mf30_{i}'] = MF30_imputer.fit_transform(globals()[f'weight30_{i}'].copy()) #for loop miss forest impputation 
  globals()[f'MSE_MF30_{i}'] = mean_squared_error(data, globals()[f'imputed_mf30_{i}']) # for loop mean_squared_error
t_end = time.time()
t30_MF = (t_end- t_start)

In [ ]:
t_start = time.time()
mse_mf30_total = 0 
MSE_MF30_1001 = 0
for i in range(0,1001):
  mse_mf30_total = mse_mf30_total + globals()[f'MSE_MF30_{i+1}']
AMSE_weight30_mf = mse_mf30_total/1000
print(f'AMSE_weight30_mf = {AMSE_weight30_mf}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight30_mf = 0.09729602599371548
เวลาที่ใช้ในการประมวลผล 0.004587888717651367 วินาที
เวลาที่ใช้ในการประมวลผล 7.646481196085611e-05 นาที


## Imputation using the KNN

In [ ]:
from sklearn import neighbors
from sklearn.impute import KNNImputer

In [ ]:
t_start = time.time()
KNN30_imputer = KNNImputer(n_neighbors = 15)
for i in range(1,1001):
  globals()[f'imputed_KNN30_{i}'] = KNN30_imputer.fit_transform(globals()[f'weight30_{i}'].copy())
  globals()[f'MSE_KNN30_{i}'] = mean_squared_error(data, globals()[f'imputed_KNN30_{i}'])
t_end = time.time()
t30_KNN = (t_end- t_start)

In [ ]:
t_start = time.time()
mse_knn30_total = 0 
MSE_KNN30_1001 = 0
for i in range(0,1001):
  mse_knn30_total = mse_knn30_total + globals()[f'MSE_KNN30_{i+1}']
AMSE_weight30_knn = mse_knn30_total/1000
print(f'AMSE_weight30_knn = {AMSE_weight30_knn}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight20_knn = 0.5635913499916201
เวลาที่ใช้ในการประมวลผล 0.004913330078125 วินาที
เวลาที่ใช้ในการประมวลผล 8.188883463541666e-05 นาที


## Imputation using the MICE

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn import svm

### LinearRegression model

In [ ]:
t_start = time.time()
lr = LinearRegression()
for i in range(1,1001):
  imp30 = IterativeImputer(estimator= lr , max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_lr30_{i}'] = imp30.fit_transform(globals()[f'weight30_{i}'].copy())
  globals()[f'MSE_lr30_{i}'] = mean_squared_error(data, globals()[f'imputed_lr30_{i}'])
t_end = time.time()
t30_LR = (t_end- t_start)

In [ ]:
t_start = time.time()
mse_lr30_total = 0 
MSE_lr30_1001 = 0
for i in range(0,1001):
  mse_lr30_total = mse_lr30_total + globals()[f'MSE_lr30_{i+1}']
AMSE_weight30_lr = mse_lr30_total/1000
print(f'AMSE_weight30_lr = {AMSE_weight30_lr}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight30_lr = 0.02564107138342625
เวลาที่ใช้ในการประมวลผล 0.005902528762817383 วินาที
เวลาที่ใช้ในการประมวลผล 9.837547938028972e-05 นาที


### Support Vector Machine model

In [ ]:
t_start = time.time()
regr = svm.SVR()
for i in range(1,1001):
  svm_svr = IterativeImputer(estimator= regr, max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_svm30_{i}'] = svm_svr.fit_transform(globals()[f'weight30_{i}'].copy())
  globals()[f'MSE_svm30_{i}'] = mean_squared_error(data, globals()[f'imputed_svm30_{i}'])
t_end = time.time()
t30_SVM = (t_end- t_start)

In [ ]:
t_start = time.time()
mse_svm30_total = 0 
MSE_svm30_1001 = 0
for i in range(0,1001):
  mse_svm30_total = mse_svm30_total + globals()[f'MSE_svm30_{i+1}']
AMSE_weight30_svm = mse_svm30_total/1000
print(f'AMSE_weight30_svm = {AMSE_weight30_svm}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight30_svm = 0.7556074545181287
เวลาที่ใช้ในการประมวลผล 0.003670215606689453 วินาที
เวลาที่ใช้ในการประมวลผล 6.117026011149088e-05 นาที


## Imputation using the EM

### impy.em

In [ ]:
!pip install impyute
import impyute as impy

In [ ]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'imputed_impy30_{i}'] = impy.em(globals()[f'weight30_{i}'].values.copy())
  globals()[f'MSE_impy30_{i}'] = mean_squared_error(data, globals()[f'imputed_impy30_{i}'])
t_end = time.time()
t30_EM = (t_end- t_start)

In [ ]:
t_start = time.time()
mse_impy30_total = 0 
MSE_impy30_1001 = 0
for i in range(0,1001):
  mse_impy30_total = mse_impy30_total + globals()[f'MSE_impy30_{i+1}']
AMSE_weight30_impy = mse_impy30_total/1000
print(f'AMSE_weight30_impy = {AMSE_weight30_impy}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight30_impy = 1.6645229318389554
เวลาที่ใช้ในการประมวลผล 0.0030002593994140625 วินาที
เวลาที่ใช้ในการประมวลผล 5.0004323323567705e-05 นาที


# 40 % random nan

In [ ]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'weight40_{i}'] = data.copy() # for loop variable name
  np.random.seed(i) # for loop seed
  n = int((153*40/100)) 
  index_nan = np.random.choice(globals()[f'weight40_{i}']['Weight'].size, n, replace=False) #globals()[name dataframe][column name].size
  globals()[f'weight40_{i}']['Weight'].ravel()[index_nan] = np.nan
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

เวลาที่ใช้ในการประมวลผล 0.22542285919189453 วินาที
เวลาที่ใช้ในการประมวลผล 0.0037570476531982424 นาที


In [ ]:
weight40_1

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,003_Creatinine,004_Uric Acid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,013_Alkaline Phos,071_HbA1c
0,2,57.0,47.4,153.0,20.2,114.0,75.0,69.0,19.3,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
1,2,24.0,53.2,157.0,21.6,103.0,69.0,78.0,21.3,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
2,2,62.0,54.4,165.0,20.0,95.0,75.0,83.0,21.6,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
3,2,35.0,56.3,161.0,21.7,95.0,57.0,72.0,22.5,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.1
4,1,46.0,NaN,170.0,25.0,133.0,77.0,107.0,29.2,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1,60.0,NaN,170.0,24.9,131.0,74.0,69.0,30.5,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.5
149,1,58.0,62.0,168.0,22.0,103.0,76.0,72.0,27.8,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.8
150,2,42.0,55.5,165.0,20.4,102.0,52.0,77.0,22.9,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.8
151,2,61.0,61.8,159.0,24.4,117.0,60.0,72.0,23.4,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.4


## Imputation using Mean

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
t_start = time.time()
mean40_imputer = SimpleImputer(strategy='mean')
for i in range(1,1001):
  globals()[f'imputed_mean40_{i}'] = mean40_imputer.fit_transform(globals()[f'weight40_{i}'].copy()) #for loop miss forest impputation 
  globals()[f'MSE_MEAN40_{i}'] = mean_squared_error(data, globals()[f'imputed_mean40_{i}']) # for loop mean_squared_error
t_end = time.time()
t40_MEAN = (t_end- t_start)

In [ ]:
t_start = time.time()
mse_mean40_total = 0 
MSE_MEAN40_1001 = 0
for i in range(0,1001):
  mse_mean40_total = mse_mean40_total + globals()[f'MSE_MEAN40_{i+1}']
AMSE_weight40_mean = mse_mean40_total/1000
print(f'AMSE_weight40_mean = {AMSE_weight40_mean}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight40_mean = 1.1932038242529084
เวลาที่ใช้ในการประมวลผล 0.0026488304138183594 วินาที
เวลาที่ใช้ในการประมวลผล 4.414717356363932e-05 นาที


## Imputation using the miss forest

In [ ]:
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
!pip install missingpy
import missingpy
from missingpy import MissForest
from sklearn.metrics import mean_squared_error

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
t_start = time.time()
MF40_imputer = MissForest()
for i in range(1,1001):
  globals()[f'imputed_mf40_{i}'] = MF40_imputer.fit_transform(globals()[f'weight40_{i}'].copy()) #for loop miss forest impputation 
  globals()[f'MSE_MF40_{i}'] = mean_squared_error(data, globals()[f'imputed_mf40_{i}']) # for loop mean_squared_error
t_end = time.time()
t40_MF = (t_end- t_start)

In [ ]:
t_start = time.time()
mse_mf40_total = 0 
MSE_MF40_1001 = 0
for i in range(0,1001):
  mse_mf40_total = mse_mf40_total + globals()[f'MSE_MF40_{i+1}']
AMSE_weight40_mf = mse_mf40_total/1000
print(f'AMSE_weight40_mf = {AMSE_weight40_mf}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight40_mf = 0.14073830175188556
เวลาที่ใช้ในการประมวลผล 0.007723093032836914 วินาที
เวลาที่ใช้ในการประมวลผล 0.00012871821721394857 นาที


## Imputation using the KNN

In [ ]:
from sklearn import neighbors
from sklearn.impute import KNNImputer

In [ ]:
t_start = time.time()
KNN40_imputer = KNNImputer(n_neighbors = 15)
for i in range(1,1001):
  globals()[f'imputed_KNN40_{i}'] = KNN40_imputer.fit_transform(globals()[f'weight40_{i}'].copy())
  globals()[f'MSE_KNN40_{i}'] = mean_squared_error(data, globals()[f'imputed_KNN40_{i}'])
t_end = time.time()
t40_KNN = (t_end- t_start)

In [ ]:
t_start = time.time()
mse_knn40_total = 0 
MSE_KNN40_1001 = 0
for i in range(0,1001):
  mse_knn40_total = mse_knn40_total + globals()[f'MSE_KNN40_{i+1}']
AMSE_weight40_knn = mse_knn40_total/1000
print(f'AMSE_weight40_knn = {AMSE_weight40_knn}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight40_knn = 0.7858749533880791
เวลาที่ใช้ในการประมวลผล 0.003650188446044922 วินาที
เวลาที่ใช้ในการประมวลผล 6.0836474100748696e-05 นาที


## Imputation using the MICE

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn import svm

### LinearRegression model

In [ ]:
t_start = time.time()
lr = LinearRegression()
for i in range(1,1001):
  imp40 = IterativeImputer(estimator= lr , max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_lr40_{i}'] = imp40.fit_transform(globals()[f'weight40_{i}'].copy())
  globals()[f'MSE_lr40_{i}'] = mean_squared_error(data, globals()[f'imputed_lr40_{i}'])
t_end = time.time()
t40_LR = (t_end- t_start)

In [ ]:
t_start = time.time()
mse_lr40_total = 0 
MSE_lr40_1001 = 0
for i in range(0,1001):
  mse_lr40_total = mse_lr40_total + globals()[f'MSE_lr40_{i+1}']
AMSE_weight40_lr = mse_lr40_total/1000
print(f'AMSE_weight40_lr = {AMSE_weight40_lr}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight40_lr = 0.0394552947816809
เวลาที่ใช้ในการประมวลผล 0.0018916130065917969 วินาที
เวลาที่ใช้ในการประมวลผล 3.152688344319661e-05 นาที


### Support Vector Machine model

In [ ]:
t_start = time.time()
regr = svm.SVR()
for i in range(1,1001):
  svm_svr = IterativeImputer(estimator= regr, max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_svm40_{i}'] = svm_svr.fit_transform(globals()[f'weight40_{i}'].copy())
  globals()[f'MSE_svm40_{i}'] = mean_squared_error(data, globals()[f'imputed_svm40_{i}'])
t_end = time.time()
t40_SVM = (t_end- t_start)

In [ ]:
t_start = time.time()
mse_svm40_total = 0 
MSE_svm40_1001 = 0
for i in range(0,1001):
  mse_svm40_total = mse_svm40_total + globals()[f'MSE_svm40_{i+1}']
AMSE_weight40_svm = mse_svm40_total/1000
print(f'AMSE_weight40_svm = {AMSE_weight40_svm}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight40_svm = 1.0432422615506154
เวลาที่ใช้ในการประมวลผล 0.008126497268676758 วินาที
เวลาที่ใช้ในการประมวลผล 0.00013544162114461264 นาที


## Imputation using the EM

### impy.em

In [ ]:
!pip install impyute
import impyute as impy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'imputed_impy40_{i}'] = impy.em(globals()[f'weight40_{i}'].values.copy())
  globals()[f'MSE_impy40_{i}'] = mean_squared_error(data, globals()[f'imputed_impy40_{i}'])
t_end = time.time()
t40_EM = (t_end- t_start)

In [ ]:
t_start = time.time()
mse_impy40_total = 0 
MSE_impy40_1001 = 0
for i in range(0,1001):
  mse_impy40_total = mse_impy40_total + globals()[f'MSE_impy40_{i+1}']
AMSE_weight40_impy = mse_impy40_total/1000
print(f'AMSE_weight40_impy = {AMSE_weight40_impy}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight40_impy = 2.273172899070936
เวลาที่ใช้ในการประมวลผล 0.0032525062561035156 วินาที
เวลาที่ใช้ในการประมวลผล 5.420843760172526e-05 นาที


# 50 % random nan

In [ ]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'weight50_{i}'] = data.copy() # for loop variable name
  np.random.seed(i) # for loop seed
  n = int((153*50/100)) 
  index_nan = np.random.choice(globals()[f'weight50_{i}']['Weight'].size, n, replace=False) #globals()[name dataframe][column name].size
  globals()[f'weight50_{i}']['Weight'].ravel()[index_nan] = np.nan
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

เวลาที่ใช้ในการประมวลผล 0.26893019676208496 วินาที
เวลาที่ใช้ในการประมวลผล 0.004482169946034749 นาที


In [ ]:
weight50_1

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,003_Creatinine,004_Uric Acid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,013_Alkaline Phos,071_HbA1c
0,2,57.0,NaN,153.0,20.2,114.0,75.0,69.0,19.3,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
1,2,24.0,53.2,157.0,21.6,103.0,69.0,78.0,21.3,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
2,2,62.0,NaN,165.0,20.0,95.0,75.0,83.0,21.6,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
3,2,35.0,56.3,161.0,21.7,95.0,57.0,72.0,22.5,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.1
4,1,46.0,NaN,170.0,25.0,133.0,77.0,107.0,29.2,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1,60.0,NaN,170.0,24.9,131.0,74.0,69.0,30.5,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.5
149,1,58.0,62.0,168.0,22.0,103.0,76.0,72.0,27.8,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.8
150,2,42.0,55.5,165.0,20.4,102.0,52.0,77.0,22.9,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.8
151,2,61.0,61.8,159.0,24.4,117.0,60.0,72.0,23.4,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.4


## Imputation using Mean

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
t_start = time.time()
mean50_imputer = SimpleImputer(strategy='mean')
for i in range(1,1001):
  globals()[f'imputed_mean50_{i}'] = mean50_imputer.fit_transform(globals()[f'weight50_{i}'].copy()) #for loop miss forest impputation 
  globals()[f'MSE_MEAN50_{i}'] = mean_squared_error(data, globals()[f'imputed_mean50_{i}']) # for loop mean_squared_error
t_end = time.time()
t50_MEAN = (t_end- t_start)

In [ ]:
t_start = time.time()
mse_mean50_total = 0 
MSE_MEAN50_1001 = 0
for i in range(0,1001):
  mse_mean50_total = mse_mean50_total + globals()[f'MSE_MEAN50_{i+1}']
AMSE_weight50_mean = mse_mean50_total/1000
print(f'AMSE_weight50_mean = {AMSE_weight50_mean}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight50_mean = 1.4941084462481766
เวลาที่ใช้ในการประมวลผล 0.00311279296875 วินาที
เวลาที่ใช้ในการประมวลผล 5.18798828125e-05 นาที


## Imputation using the miss forest

In [ ]:
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
!pip install missingpy
import missingpy
from missingpy import MissForest
from sklearn.metrics import mean_squared_error

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
t_start = time.time()
MF50_imputer = MissForest()
for i in range(1,1001):
  globals()[f'imputed_mf50_{i}'] = MF50_imputer.fit_transform(globals()[f'weight50_{i}'].copy()) #for loop miss forest impputation 
  globals()[f'MSE_MF50_{i}'] = mean_squared_error(data, globals()[f'imputed_mf50_{i}']) # for loop mean_squared_error
t_end = time.time()
t50_MF = (t_end- t_start)

In [ ]:
t_start = time.time()
mse_mf50_total = 0 
MSE_MF50_1001 = 0
for i in range(0,1001):
  mse_mf50_total = mse_mf50_total + globals()[f'MSE_MF50_{i+1}']
AMSE_weight50_mf = mse_mf50_total/1000
print(f'AMSE_weight50_mf = {AMSE_weight50_mf}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight50_mf = 0.19266679207214688
เวลาที่ใช้ในการประมวลผล 0.02689528465270996 วินาที
เวลาที่ใช้ในการประมวลผล 0.00044825474421183266 นาที


## Imputation using the KNN

In [ ]:
from sklearn import neighbors
from sklearn.impute import KNNImputer

In [ ]:
t_start = time.time()
KNN50_imputer = KNNImputer(n_neighbors = 15)
for i in range(1,1001):
  globals()[f'imputed_KNN50_{i}'] = KNN30_imputer.fit_transform(globals()[f'weight50_{i}'].copy())
  globals()[f'MSE_KNN50_{i}'] = mean_squared_error(data, globals()[f'imputed_KNN50_{i}'])
t_end = time.time()
t50_KNN = (t_end- t_start)

In [ ]:
t_start = time.time()
mse_knn50_total = 0 
MSE_KNN50_1001 = 0
for i in range(0,1001):
  mse_knn50_total = mse_knn30_total + globals()[f'MSE_KNN50_{i+1}']
AMSE_weight50_knn = mse_knn30_total/1000
print(f'AMSE_weight50_knn = {AMSE_weight50_knn}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight50_knn = 0.5635913499916201
เวลาที่ใช้ในการประมวลผล 0.00854945182800293 วินาที
เวลาที่ใช้ในการประมวลผล 0.00014249086380004882 นาที


## Imputation using the MICE

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn import svm

### LinearRegression model

In [ ]:
t_start = time.time()
lr = LinearRegression()
for i in range(1,1001):
  imp50 = IterativeImputer(estimator= lr , max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_lr50_{i}'] = imp50.fit_transform(globals()[f'weight50_{i}'].copy())
  globals()[f'MSE_lr50_{i}'] = mean_squared_error(data, globals()[f'imputed_lr50_{i}'])
t_end = time.time()
t50_LR = (t_end- t_start)

In [ ]:
t_start = time.time()
mse_lr50_total = 0 
MSE_lr50_1001 = 0
for i in range(0,1001):
  mse_lr50_total = mse_lr50_total + globals()[f'MSE_lr50_{i+1}']
AMSE_weight50_lr = mse_lr50_total/1000
print(f'AMSE_weight50_lr = {AMSE_weight50_lr}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight50_lr = 0.057924093090379665
เวลาที่ใช้ในการประมวลผล 0.0012845993041992188 วินาที
เวลาที่ใช้ในการประมวลผล 2.1409988403320312e-05 นาที


### Support Vector Machine model

In [ ]:
t_start = time.time()
regr = svm.SVR()
for i in range(1,1001):
  svm_svr = IterativeImputer(estimator= regr, max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_svm50_{i}'] = svm_svr.fit_transform(globals()[f'weight50_{i}'].copy())
  globals()[f'MSE_svm50_{i}'] = mean_squared_error(data, globals()[f'imputed_svm50_{i}'])
t_end = time.time()
t50_SVM = (t_end- t_start)

In [ ]:
t_start = time.time()
mse_svm50_total = 0 
MSE_svm50_1001 = 0
for i in range(0,1001):
  mse_svm50_total = mse_svm50_total + globals()[f'MSE_svm50_{i+1}']
AMSE_weight50_svm = mse_svm50_total/1000
print(f'AMSE_weight50_svm = {AMSE_weight50_svm}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight50_svm = 1.3405980838715348
เวลาที่ใช้ในการประมวลผล 0.0038013458251953125 วินาที
เวลาที่ใช้ในการประมวลผล 6.335576375325521e-05 นาที


## Imputation using the EM

### impy.em

In [ ]:
!pip install impyute
import impyute as impy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'imputed_impy50_{i}'] = impy.em(globals()[f'weight50_{i}'].values.copy())
  globals()[f'MSE_impy50_{i}'] = mean_squared_error(data, globals()[f'imputed_impy50_{i}'])
t_end = time.time()
t50_EM = (t_end- t_start)

In [ ]:
t_start = time.time()
mse_impy50_total = 0 
MSE_impy50_1001 = 0
for i in range(0,1001):
  mse_impy50_total = mse_impy50_total + globals()[f'MSE_impy50_{i+1}']
AMSE_weight50_impy = mse_impy50_total/1000
print(f'AMSE_weight50_impy = {AMSE_weight50_impy}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight50_impy = 2.8751633547878694
เวลาที่ใช้ในการประมวลผล 0.002830028533935547 วินาที
เวลาที่ใช้ในการประมวลผล 4.716714223225912e-05 นาที


# 60 % random nan

In [ ]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'weight60_{i}'] = data.copy() # for loop variable name
  np.random.seed(i) # for loop seed
  n = int((153*60/100)) 
  index_nan = np.random.choice(globals()[f'weight60_{i}']['Weight'].size, n, replace=False) #globals()[name dataframe][column name].size
  globals()[f'weight60_{i}']['Weight'].ravel()[index_nan] = np.nan
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

เวลาที่ใช้ในการประมวลผล 0.2926139831542969 วินาที
เวลาที่ใช้ในการประมวลผล 0.004876899719238281 นาที


In [ ]:
weight60_1

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,003_Creatinine,004_Uric Acid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,013_Alkaline Phos,071_HbA1c
0,2,57.0,NaN,153.0,20.2,114.0,75.0,69.0,19.3,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
1,2,24.0,53.2,157.0,21.6,103.0,69.0,78.0,21.3,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
2,2,62.0,NaN,165.0,20.0,95.0,75.0,83.0,21.6,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
3,2,35.0,56.3,161.0,21.7,95.0,57.0,72.0,22.5,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.1
4,1,46.0,NaN,170.0,25.0,133.0,77.0,107.0,29.2,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1,60.0,NaN,170.0,24.9,131.0,74.0,69.0,30.5,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.5
149,1,58.0,62.0,168.0,22.0,103.0,76.0,72.0,27.8,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.8
150,2,42.0,55.5,165.0,20.4,102.0,52.0,77.0,22.9,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.8
151,2,61.0,NaN,159.0,24.4,117.0,60.0,72.0,23.4,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.4


## Imputation using Mean

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
t_start = time.time()
mean60_imputer = SimpleImputer(strategy='mean')
for i in range(1,1001):
  globals()[f'imputed_mean60_{i}'] = mean60_imputer.fit_transform(globals()[f'weight60_{i}'].copy()) #for loop miss forest impputation 
  globals()[f'MSE_MEAN60_{i}'] = mean_squared_error(data, globals()[f'imputed_mean60_{i}']) # for loop mean_squared_error
t_end = time.time()
t60_MEAN = (t_end- t_start)

In [ ]:
t_start = time.time()
mse_mean60_total = 0 
MSE_MEAN60_1001 = 0
for i in range(0,1001):
  mse_mean60_total = mse_mean60_total + globals()[f'MSE_MEAN60_{i+1}']
AMSE_weight60_mean = mse_mean60_total/1000
print(f'AMSE_weight60_mean = {AMSE_weight60_mean}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight60_mean = 1.803050654777136
เวลาที่ใช้ในการประมวลผล 0.0047168731689453125 วินาที
เวลาที่ใช้ในการประมวลผล 7.861455281575521e-05 นาที


## Imputation using the miss forest

In [ ]:
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
!pip install missingpy
import missingpy
from missingpy import MissForest
from sklearn.metrics import mean_squared_error

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
t_start = time.time()
MF60_imputer = MissForest()
for i in range(1,1001):
  globals()[f'imputed_mf60_{i}'] = MF60_imputer.fit_transform(globals()[f'weight60_{i}'].copy()) #for loop miss forest impputation 
  globals()[f'MSE_MF60_{i}'] = mean_squared_error(data, globals()[f'imputed_mf60_{i}']) # for loop mean_squared_error
t_end = time.time()
t60_MF = (t_end- t_start)

In [ ]:
t_start = time.time()
mse_mf60_total = 0 
MSE_MF60_1001 = 0
for i in range(0,1001):
  mse_mf60_total = mse_mf60_total + globals()[f'MSE_MF60_{i+1}']
AMSE_weight60_mf = mse_mf60_total/1000
print(f'AMSE_weight60_mf = {AMSE_weight60_mf}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight60_mf = 0.2717420365477631
เวลาที่ใช้ในการประมวลผล 0.0040171146392822266 วินาที
เวลาที่ใช้ในการประมวลผล 6.695191065470378e-05 นาที


## Imputation using the KNN

In [ ]:
from sklearn import neighbors
from sklearn.impute import KNNImputer

In [ ]:
t_start = time.time()
KNN60_imputer = KNNImputer(n_neighbors = 15)
for i in range(1,1001):
  globals()[f'imputed_KNN60_{i}'] = KNN60_imputer.fit_transform(globals()[f'weight60_{i}'].copy())
  globals()[f'MSE_KNN60_{i}'] = mean_squared_error(data, globals()[f'imputed_KNN60_{i}'])
t_end = time.time()
t60_KNN = (t_end- t_start)

In [ ]:
t_start = time.time()
mse_knn60_total = 0 
MSE_KNN60_1001 = 0
for i in range(0,1001):
  mse_knn60_total = mse_knn60_total + globals()[f'MSE_KNN60_{i+1}']
AMSE_weight60_knn = mse_knn60_total/1000
print(f'AMSE_weight60_knn = {AMSE_weight60_knn}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight60_knn = 1.2946601193229423
เวลาที่ใช้ในการประมวลผล 0.0073032379150390625 วินาที
เวลาที่ใช้ในการประมวลผล 0.00012172063191731771 นาที


## Imputation using the MICE

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn import svm

### LinearRegression model

In [ ]:
t_start = time.time()
lr = LinearRegression()
for i in range(1,1001):
  imp60 = IterativeImputer(estimator= lr , max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_lr60_{i}'] = imp60.fit_transform(globals()[f'weight60_{i}'].copy())
  globals()[f'MSE_lr60_{i}'] = mean_squared_error(data, globals()[f'imputed_lr60_{i}'])
t_end = time.time()
t60_LR = (t_end- t_start)

In [ ]:
t_start = time.time()
mse_lr60_total = 0 
MSE_lr60_1001 = 0
for i in range(0,1001):
  mse_lr60_total = mse_lr60_total + globals()[f'MSE_lr60_{i+1}']
AMSE_weight60_lr = mse_lr60_total/1000
print(f'AMSE_weight60_lr = {AMSE_weight60_lr}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight60_lr = 0.08473347198456993
เวลาที่ใช้ในการประมวลผล 0.004246234893798828 วินาที
เวลาที่ใช้ในการประมวลผล 7.07705815633138e-05 นาที


### Support Vector Machine model

In [ ]:
t_start = time.time()
regr = svm.SVR()
for i in range(1,1001):
  svm_svr = IterativeImputer(estimator= regr, max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_svm60_{i}'] = svm_svr.fit_transform(globals()[f'weight60_{i}'].copy())
  globals()[f'MSE_svm60_{i}'] = mean_squared_error(data, globals()[f'imputed_svm60_{i}'])
t_end = time.time()
t60_SVM = (t_end- t_start)

In [ ]:
t_start = time.time()
mse_svm60_total = 0 
MSE_svm60_1001 = 0
for i in range(0,1001):
  mse_svm60_total = mse_svm60_total + globals()[f'MSE_svm60_{i+1}']
AMSE_weight60_svm = mse_svm60_total/1000
print(f'AMSE_weight60_svm = {AMSE_weight60_svm}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight60_svm = 1.6591846188280894
เวลาที่ใช้ในการประมวลผล 0.002452850341796875 วินาที
เวลาที่ใช้ในการประมวลผล 4.088083902994792e-05 นาที


## Imputation using the EM

### impy.em

In [ ]:
!pip install impyute
import impyute as impy

In [ ]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'imputed_impy60_{i}'] = impy.em(globals()[f'weight60_{i}'].values.copy())
  globals()[f'MSE_impy60_{i}'] = mean_squared_error(data, globals()[f'imputed_impy60_{i}'])
t_end = time.time()
t60_EM = (t_end- t_start)

In [ ]:
t_start = time.time()
mse_impy60_total = 0 
MSE_impy60_1001 = 0
for i in range(0,1001):
  mse_impy60_total = mse_impy60_total + globals()[f'MSE_impy60_{i+1}']
AMSE_weight60_impy = mse_impy60_total/1000
print(f'AMSE_weight60_impy = {AMSE_weight60_impy}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight60_impy = 3.4950180315820423
เวลาที่ใช้ในการประมวลผล 0.0019898414611816406 วินาที
เวลาที่ใช้ในการประมวลผล 3.316402435302734e-05 นาที


# AMSE

In [ ]:
amse_value = {'Method':['MEAN','Miss Forest','KNN','MICE-LR','MICE-SVM','EM'],'10%':[AMSE_weight10_mean,AMSE_weight10_mf,AMSE_weight10_knn,AMSE_weight10_lr,AMSE_weight10_svm,AMSE_weight10_impy],
      '20%':[AMSE_weight20_mean,AMSE_weight20_mf,AMSE_weight20_knn,AMSE_weight20_lr,AMSE_weight20_svm,AMSE_weight20_impy],
      '30%':[AMSE_weight30_mean,AMSE_weight30_mf,AMSE_weight30_knn,AMSE_weight30_lr,AMSE_weight30_svm,AMSE_weight30_impy],
      '40%':[AMSE_weight40_mean,AMSE_weight40_mf,AMSE_weight40_knn,AMSE_weight40_lr,AMSE_weight40_svm,AMSE_weight40_impy],
      '50%':[AMSE_weight50_mean,AMSE_weight50_mf,AMSE_weight50_knn,AMSE_weight50_lr,AMSE_weight60_svm,AMSE_weight50_impy]
      ,'60%':[AMSE_weight60_mean,AMSE_weight60_mf,AMSE_weight60_knn,AMSE_weight60_lr,AMSE_weight60_svm,AMSE_weight60_impy]}

In [ ]:
data_AMSE = pd.DataFrame.from_dict(amse_value)
data_AMSE

# เวลาทั้งหมดในการรัน

## second 

In [ ]:
data_t_sec = {'Method':['MEAN','Miss Forest','KNN','MICE-LR','MICE-SVM','EM'],'10%':[t10_MEAN,t10_MF,t10_KNN,t10_LR,t10_SVM,t10_EM],
      '20%':[t20_MEAN,t20_MF,t20_KNN,t20_LR,t20_SVM,t20_EM],'30%':[t30_MEAN,t30_MF,t30_KNN,t30_LR,t30_SVM,t30_EM],'40%':[t40_MEAN,t40_MF,t40_KNN,t40_LR,t40_SVM,t40_EM],
      '50%':[t50_MEAN,t50_MF,t50_KNN,t50_LR,t50_SVM,t50_EM],'60%':[t60_MEAN,t60_MF,t60_KNN,t60_LR,t60_SVM,t60_EM]}

In [ ]:
data_second = pd.DataFrame.from_dict(data_t_sec)
data_second

,Method,10%,20%,30%,40%,50%,60%
0,MEAN,5.011079,5.654908,5.666173,5.077938,5.346818,5.787479
1,Miss Forest,1101.781403,1069.644502,1043.665598,1001.200485,963.178606,950.340620
2,KNN,7.465795,8.150183,14.855747,8.949518,21.180088,19.111311
3,MICE-LR,72.775453,72.915364,69.912066,76.231953,66.672497,67.996151
4,MICE-SVM,150.511730,151.056799,150.705075,147.752676,142.320703,146.515236
5,EM,13.240064,24.694545,29.484782,38.509766,48.326113,49.508676


## minutes

In [ ]:
data_t_min = {'Method':['MEAN','Miss Forest','KNN','MICE-LR','MICE-SVM','EM'],'10%':[t10_MEAN/60,t10_MF/60,t10_KNN/60,t10_LR/60,t10_SVM/60,t10_EM/60],
      '20%':[t20_MEAN/60,t20_MF/60,t20_KNN/60,t20_LR/60,t20_SVM/60,t20_EM/60],'30%':[t30_MEAN/60,t30_MF/60,t30_KNN/60,t30_LR/60,t30_SVM/60,t30_EM/60]
      ,'40%':[t40_MEAN/60,t40_MF/60,t40_KNN/60,t40_LR/60,t40_SVM/60,t40_EM/60],'50%':[t50_MEAN/60,t50_MF/60,t50_KNN/60,t50_LR/60,t50_SVM/60,t50_EM/60],
      '60%':[t60_MEAN/60,t60_MF/60,t60_KNN/60,t60_LR/60,t60_SVM/60,t60_EM/60]}

In [ ]:
data_minutes = pd.DataFrame.from_dict(data_t_min)
data_minutes

,Method,10%,20%,30%,40%,50%,60%
0,MEAN,0.083518,0.094248,0.094436,0.084632,0.089114,0.096458
1,Miss Forest,18.363023,17.827408,17.394427,16.686675,16.052977,15.839010
2,KNN,0.124430,0.135836,0.247596,0.149159,0.353001,0.318522
3,MICE-LR,1.212924,1.215256,1.165201,1.270533,1.111208,1.133269
4,MICE-SVM,2.508529,2.517613,2.511751,2.462545,2.372012,2.441921
5,EM,0.220668,0.411576,0.491413,0.641829,0.805435,0.825145
